# Importy:

In [1]:
# import os
# import json
# import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
# sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
# import torchvision
# from torchvision.utils import save_image
# from torchvision.datasets import FashionMNIST
# from torchvision.transforms import v2
# from torchvision.datasets import ImageFolder
# from torch.nn.functional import one_hot

# from pytorch_fid import fid_score

# from collections import Counter
from sklearn.model_selection import train_test_split

/tmp/ipykernel_5149/2549380766.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
/home/kuba/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [3]:
import transformers
from transformers import *
from datasets import load_dataset
import evaluate
from tqdm import tqdm

/home/kuba/.local/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


# Wczytywanie danych

In [4]:
PATH = '../../data'
reviews = pd.read_csv(f"{PATH}/train_data.csv")
reviews.head()

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3


In [5]:
reviews['rating'].value_counts()

rating
4    7243
3    4831
2    1747
1    1434
0    1137
Name: count, dtype: int64

Z komórki wyżej wynika, że dane treningowe są niezbilansowane. \
Znamy 2 główne metody "naprawy" tego problemu: undersampling i oversampling. \
Zdecydowaliśmy się zastosować metodę undersamplingu, w nadziei, że prawie 1200 wierszy dla każdej z klas wystarczy.

In [6]:
X = reviews.filter(['review'])
y = reviews.filter(['rating'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [8]:
from imblearn.under_sampling import RandomUnderSampler

In [9]:
sampler = RandomUnderSampler(sampling_strategy="not minority")
resampled_data, resampled_classes = sampler.fit_resample(X_train, y_train)

In [10]:
resampled_classes.value_counts()

rating
0         790
1         790
2         790
3         790
4         790
Name: count, dtype: int64

Najpierw podzieliliśmy początkowy zbiór testowy na nowe dane testowe i walidacyjne, a następnie zbilansowaliśmy dane testowe otrzymane w wyniku podziału. Kolejność tych operacji jest ważna, ponieważ chcemy, aby zbiór walidacyjny nie zawierał żadnych zmian, natomiast zbiór testowy został zbilansowany, aby 

In [11]:
resampled = pd.merge(resampled_data, resampled_classes, left_index=True, right_index=True)

In [12]:
resampled.head()

,review,rating
4635,disappointed service there-the restaurant open...,0
13177,"run thought picking cheap hotel save lot, not,...",0
5778,"rating suspect, hotel sofitel june 6-7 power w...",0
3526,"words, ok stay riu hotel punta cana, twice sta...",0
982,n't fooled price location grade room damrak ho...,0


In [13]:
resampled['rating'].value_counts()

rating
0    790
1    790
2    790
3    790
4    790
Name: count, dtype: int64

# Modele

## Model Rekurencyjny

In [14]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)

In [15]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, emb_weights, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

In [16]:
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(emb_weights)
tokenizer = gensim_model.wv.key_to_index

In [17]:
lstm_model = LSTMRegressor(100, 100, 1, 5, emb_weights).to(device)
lstm_model

LSTMRegressor(
  (embeddings): Embedding(71290, 100)
  (lstm): LSTM(100, 100)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)

## Gotowe modele z biblioteki sklearn

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score

In [19]:
pipelineMNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

pipelineCNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ComplementNB())
])

TypeError: Can't instantiate abstract class Pipeline with abstract methods _forward, _sanitize_parameters, postprocess, preprocess

In [ ]:
pipelineMNB.fit(resampled_data, resampled_classes)
predict1 = pipelineMNB.predict(X_test)
print(accuracy_score(y_test, predict1))